In [ ]:
import pandas as pd

In [ ]:
from transformers import DistilBertTokenizerFast

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

True

In [ ]:
data_pre_processing_folder =  "../data/pre-processing/"
data_pre_processing_name = "data.csv"
label_pre_processing_name = "label.csv"
model_path =  "../models/classifier/"
tokenizer_path =  "../models/tokenizer/"

In [ ]:
data = pd.read_csv(data_pre_processing_folder + data_pre_processing_name)["text"]
label = pd.read_csv(data_pre_processing_folder + label_pre_processing_name)["label"]

In [ ]:
epochs = 3
num_labels = label.nunique()
lr = 5e-5

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, label, stratify=label, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=label, test_size=0.25, random_state=1)

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
X_train = tokenizer(X_train.to_list(), truncation=True, padding=True)
X_val = tokenizer(X_val.to_list(), truncation=True, padding=True)
X_test = tokenizer(X_test.to_list(), truncation=True, padding=True)

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(X_train, y_train.to_list())
val_dataset = Dataset(X_val, y_val.to_list())
test_dataset = Dataset(X_test, y_test.to_list())

In [ ]:
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)
model.to(device)
model.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [ ]:
optim = AdamW(model.parameters(), lr=lr)

for epoch in range(epochs):
    for batch in tqdm(train_loader):
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()
        optim.zero_grad()
model.eval()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1937 [00:00<?, ?it/s]

  0%|          | 0/1937 [00:00<?, ?it/s]

  0%|          | 0/1937 [00:00<?, ?it/s]

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:

model.save_pretrained(model_path, from_pt=True)
tokenizer.save_pretrained(tokenizer_path, from_pt=True) 

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained(model_path)
model = model.to(device)

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained(tokenizer_path)

In [ ]:
val_loader = DataLoader(val_dataset, batch_size=16)

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
import torch.nn.functional as F
from sklearn.metrics import f1_score

In [ ]:
def validation(validation_dataloader):
  with torch.no_grad():
    loss_val_list = []
    preds_list = []
    accuracy_list = []
    accuracy_sum = 0
    for batch in tqdm(validation_dataloader):
      #print(batch.keys())
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)

      outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
      loss = outputs[0]
      logits = F.softmax(outputs[1], dim=1)   # Taking the softmax of output
      _,preds = torch.max(logits, dim=1)      # Taking the predictions of our batch
      acc = accuracy(logits,labels)           # Calculating the accuracy of current batch
      accuracy_sum += acc                     # Taking sum of all the accuracies of all the batches. This sum will be divided by batch length to get mean accuracy for validation dataset

      loss_val_list.append(loss)
      preds_list.append(preds)
      accuracy_list.append(acc)

  mean_accuracy = accuracy_sum / len(validation_dataloader)
  return mean_accuracy, preds_list

In [ ]:
mean_accuracy, preds_list = validation(val_loader)

In [ ]:
import numpy as np

In [ ]:
y_pred = np.concatenate( list(i.cpu().numpy() for i in preds_list), axis=0 )

In [ ]:
f1_score(y_val, y_pred, average='weighted'), f1_score(y_val, y_pred, average='micro'), f1_score(y_val, y_pred, average='macro')

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=16)

In [ ]:
mean_accuracy, preds_list = validation(test_loader)

In [ ]:
y_pred = np.concatenate( list(i.cpu().numpy() for i in preds_list), axis=0 )

In [ ]:
f1_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred, average='micro'), f1_score(y_test, y_pred, average='macro')